In [4]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

import sys 
sys.path.append('.')


from cgiar.utils import get_dir
from cgiar.model import ResNetMultipleMLP, XCITMultipleMLP
from cgiar.data import CGIARDataset_V4, augmentations

# reduce font size of plots
plt.rcParams.update({'font.size': 8})

In [5]:
SEED=42
LR=1e-4
EPOCHS=35
IMAGE_SIZE=224
INITIAL_SIZE=512
TRAIN_BATCH_SIZE=128
TEST_BATCH_SIZE=256
HIDDEN_SIZE=128
NUM_FOLDS=5
NUM_VIEWS=20

In [6]:
# ensure reproducibility
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

In [7]:
DATA_DIR = get_dir('data/')
ARTIFACTS = get_dir('solutions/v10/#3/')

In [8]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    augmentations["RandomEqualize"],
    augmentations["RandomAffine"],
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Test Submission

In [9]:
# Load test data frame from csv
X_test = pd.read_csv(DATA_DIR / 'Test.csv')

test_images = CGIARDataset_V4.load_images(X_test, DATA_DIR / "test", INITIAL_SIZE)
test_images = dict([test_images[idx] for idx in range(len(test_images))])

  0%|          | 0/8663 [00:00<?, ?it/s]

100%|██████████| 8663/8663 [03:02<00:00, 47.54it/s] 


In [10]:
# Construct test dataset and dataloader
test_dataset = CGIARDataset_V4(
    images=test_images,
    num_views=NUM_VIEWS,
    transform=transform,
    features=X_test,
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=TEST_BATCH_SIZE, 
    shuffle=False
)

In [11]:
weights = {
    0: 1.0,
    # 2: 1.0,
    # 3: 1.0,
    # 4: 1.0
}

In [12]:
def get_models(device):
    
    models = {}
    
    for i in weights.keys():
        # initialize the model
        model = XCITMultipleMLP(
            model_name="xcit_nano_12_p16_224",
            pretrained=True,
            num_mlps=4,
            hidden_size=HIDDEN_SIZE
        )
        
        # load the model weights
        model.load_state_dict(torch.load(ARTIFACTS / f"model_fold_{i}.pth", map_location=device))
        
        # move model to gpu
        model = model.to(device)
        
        # set model to evaluation mode
        model.eval()
        
        models[i] = model
        
        # activate all dropout layers
        # for module in model.modules():
        #     if isinstance(module, torch.nn.Dropout):
        #         module.train()
        
    return models

In [13]:
test_ensemble_predictions = []


with torch.cuda.device(0):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    models = get_models(device)
    
    with torch.no_grad():
        for ids, images_list, growth_stage, season, _ in tqdm(test_loader):
            
            outputs = torch.zeros(len(ids)).to(device)
            
            for idx in weights.keys():
            
                # average predictions from all the views
                outputs += weights[idx] * torch.stack([models[idx]((
                    growth_stage.to(device).squeeze(),
                    season.to(device).squeeze(),
                    images.to(device)
                )) for images in images_list]).mean(dim=0)
            
            # get predictions from all the folds
            outputs = outputs.tolist()
            test_ensemble_predictions.extend(list(zip(ids, outputs)))

# load the sample submission file and update the extent column with the predictions
submission_df = pd.read_csv('data/SampleSubmission.csv')

# update the extent column with the predictions
submission_df['extent'] = submission_df['ID'].map(dict(test_ensemble_predictions))

# save the submission file and trained model
submission_df.to_csv('submission_ensemble.csv', index=False)

100%|██████████| 34/34 [07:38<00:00, 13.49s/it]


## Train Predictions (Optional)

In [ ]:
# Load test data frame from csv
df_train = pd.read_csv(DATA_DIR / 'Train.csv')
X_train = df_train.drop(columns=['extent'], axis=1)
y_train = df_train['extent']

train_images = CGIARDataset_V4.load_images(X_train, DATA_DIR / "train", INITIAL_SIZE)
train_images = dict([train_images[idx] for idx in range(len(train_images))])

  0%|          | 0/26068 [00:00<?, ?it/s]

100%|██████████| 26068/26068 [09:13<00:00, 47.07it/s] 


In [ ]:
# Construct test dataset and dataloader
train_dataset = CGIARDataset_V4(
    transform=transform,
    labels=y_train,
    features=X_train,
    images=train_images,
    num_views=NUM_VIEWS,
)
train_loader = DataLoader(
    train_dataset, 
    batch_size=TEST_BATCH_SIZE, 
    shuffle=False
)

In [ ]:
train_ensemble_predictions = []


with torch.cuda.device(1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    models = get_models(device)
    
    with torch.no_grad():
        for ids, images_list, growth_stage, season, _ in tqdm(train_loader):
            
            outputs = torch.zeros(len(ids)).to(device)
            
            for idx in weights.keys():
            
                # average predictions from all the views
                outputs += weights[idx] * torch.stack([models[idx]((
                    growth_stage.to(device).squeeze(),
                    season.to(device).squeeze(),
                    images.to(device)
                )) for images in images_list]).mean(dim=0)
            
            # get predictions from all the folds
            outputs = outputs.tolist()
            train_ensemble_predictions.extend(list(zip(ids, outputs)))

# update the extent column with the predictions
df_train['predicted_extent'] = df_train['ID'].map(dict(train_ensemble_predictions))

# save the submission file and trained model
df_train.to_csv('train_predictions_ensemble.csv', index=False)

100%|██████████| 102/102 [13:27<00:00,  7.91s/it]
